<a href="https://colab.research.google.com/github/dlsun/pods/blob/master/05-Regression-Models/5.3%20Categorical%20Variables%20in%20Regression%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.3 Categorical Features in Regression Models

So far, we have fit linear and $k$-nearest neighbors regression models to data where all of the features are quantitative. But what if all or some of the features are categorical? In theory, the solution is simple: we simply transform the categorical variables into quantitative variables using dummy (i.e., one-hot) encoding, following the process in Chapter 3. However, in practice, some care is needed to ensure that the categorical variables are transformed in a consistent way between the training and the test data.

In [0]:
import pandas as pd
df_housing = pd.read_csv("http://dlsun.github.io/pods/data/AmesHousing.txt", sep="\t")
df_housing.head()

## One Categorical Feature

Let's develop some intuition about the predictions that a regression model will make when there is a single categorical feature. First, suppose we train a linear regression model to predict house price from the neighborhood the house is in.

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

X = df_housing[["Neighborhood"]] # need 2D array for sklearn
y = df_housing["SalePrice"]

enc = OneHotEncoder()
X_dummies = enc.fit_transform(X)

model = LinearRegression()
model.fit(X_dummies, y)

A regression model with just a single feature, **Neighborhood**, will predict the same price for all houses in the same neighborhood. What is that predicted value? We can obtain it by applying the `OneHotEncoder` to a list of the unique neighborhoods in the data set and passing this to `model.predict()`.

One way to obtain a list of the unique neighborhoods is inside the encoder itself, under the attribute `.categories_`. We convert this to a 2D-array to be compatible with scikit-learn.

In [0]:
X_test = pd.Series(enc.categories_[0], name="Neighborhood").to_frame()
X_test

In [0]:
model.predict(enc.transform(X_test))

It is a bit hard to tell which prediction corresponds to which neighborhood. Let's put these numbers into a `Series`, indexed by the neighborhood.

In [0]:
pd.Series(
    model.predict(enc.transform(X_test)),
    index=X_test["Neighborhood"]
)

Could we have obtained these predictions some other way, without going through the trouble of fitting a linear regression model? Intuitively, if all we knew about a house was the neighborhood it was in, we would predict the average price of houses in that neighborhood.

In [0]:
df_housing.groupby("Neighborhood")["SalePrice"].mean()

These numbers match the predictions from our linear regression model exactly. Linear regression simply predicts the average price in each neighborhood. 

To see this mathematically, recall that linear regression minimizes the total squared distance between the observed price and the predicted price:

$$ \text{sum of } (\text{price} - \widehat{\text{price}})^2. $$

After we expand the **Neighborhood** column into 28 dummy variables (e.g., $I\{ \text{Blmngtn} \}$, $I\{ \text{Blueste} \}$, etc.), one for each neighborhood, we can write the predicted price in the linear regression model as 

$$ \widehat{\text{price}} = c_1 I\{ \text{Blmngtn} \} + c_2 I\{ \text{Blueste} \} + \ldots + c_{28} I\{ \text{Veenker} \}. $$

(For simplicity, we have omitted the intercept term $b$.)

Now, consider a house in Bloomington Heights, for which $I\{ \text{Blmngtn} \} = 1$ and all of the other dummy variables $I\{ \text{Blueste} \} = \ldots = I\{ \text{Veenker} \} = 0$. Then, $\widehat{\text{price}}$ for a house in Bloomington Heights is $c_1$. Likewise, $\widehat{\text{price}}$ for a house in Bluestem is $c_2$. And so forth.

Now, we can reframe linear regression as learning the values $c_1, c_2, \ldots, c_{28}$ that minimize

$$ \text{sum of } (\text{price} - \widehat{\text{price}})^2 = \underbrace{\text{sum of } (\text{price} - c_1)^2}_{\text{over houses in Blmngtn}} + \underbrace{\text{sum of } (\text{price} - c_2)^2}_{\text{over houses in Blueste}} + \ldots + \underbrace{\text{sum of } (\text{price} - c_{28})^2}_{\text{over houses in Veenker}}. $$ 

We saw in Chapter 3 that the value of $c$ that mimimizes the $\text{sum of } (\text{price} - c)^2$ is the mean of the prices. So $\hat c_1$ will be the average price of houses in Bloomington Heights, $\hat c_2$ the average price of houses in Bluestem, and so on. Since $\hat c_1, \hat c_2, \ldots, \hat c_{28}$ are also the predicted values for each neighborhood, this shows that linear regression will predict the average label in each category when there is only one categorical variable in the model.

Exercise 1 in this lesson asks you to investigate what $k$-nearest neighbors regression does in the same situation.

## Mixing Quantitative and Categorical Features

In general, we want to fit machine learning models that use a mix of both categorical and quantitative features. In this situation, we will want to apply the `OneHotEncoder` to only the categorical features. Scikit-learn provides a `ColumnTransformer` that allows us to selectively apply transformations to certain columns.

For example, suppose we want to fit a $k$-nearest neighbors model to predict house price from quantitative features (square footage, number of bedrooms, number of full bathrooms) and categorical features (neighborhood, building type). We can use a `ColumnTransformer` to standardize the quantitative features and one-hot encode the categorical features.

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = make_column_transformer(
    (StandardScaler(), ["Gr Liv Area", "Bedroom AbvGr", "Full Bath"]),
    (OneHotEncoder(), ["Neighborhood", "Bldg Type"]),
    remainder="drop"  # all other columns in X will be dropped.
)
ct

Next, we integrate this `ColumnTransformer` into a pipeline (refer to the previous lesson) with the `KNeighborsRegressor` model.

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor

pipeline = make_pipeline(
    ct,
    KNeighborsRegressor(n_neighbors=10)
)

pipeline.fit(X=df_housing[["Gr Liv Area", "Bedroom AbvGr", "Full Bath",
                           "Neighborhood", "Bldg Type"]], 
             y=df_housing["SalePrice"])

Now, if we wanted to use this model to predict the price of a 3BR/2BA, 1700 sqft single-family house in Bloomington Heights, we could create a `Series` with this information, and call `pipeline.predict()` on a 2D-array with this single row.

In [0]:
x_test = pd.Series()
x_test["Gr Liv Area"] = 1700
x_test["Bedroom AbvGr"] = 3
x_test["Full Bath"] = 2
x_test["Neighborhood"] = "Blmngtn"
x_test["Bldg Type"] = "1Fam"

pipeline.predict(X=pd.DataFrame([x_test]))

So this house is predicted to cost $251,550.

## Exercises

1\. Using the Ames data set, build a $10$-nearest neighbors model to predict house price using **Neighborhood** as the only feature. How do the predictions compare with just using the mean house price of each neighborhood? If there are any discrepancies, can you explain why?

2\. In the example from the lesson, we standardized the quantitative features and one-hot encoded the categorical features in parallel. This means that the dummy variables were not standardized before being passed into the $10$-nearest neighbors model. How would you modify the pipeline so that *all* of the variables are standardized?

(_Hint:_ You may find the `remainder="passthrough"` option of `ColumnTransformer` helpful.)

3\. Using the tips data set (http://dlsun.github.io/pods/data/tips.csv ), use a $5$-nearest neighbors model to predict how much a male diner will tip on a Sunday bill of \$40.00.